Taking Inpuut

In [27]:
pdf_path=input("enter pdf path:")
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(pdf_path)
doc=loader.load()
doc

[Document(metadata={'producer': 'Skia/PDF m139 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'test 1', 'source': 'test 1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="Java  OOP(Object  Oriented  Programming)  \nConcepts\n \nLast  Updated  :  25  Jun,  2025  \n         \nJava  Object-Oriented  Programming  (OOPs)  is  a  fundamental  concept  in  \nJava\n \nthat\n \nevery\n \ndeveloper\n \nmust\n \nunderstand.\n \nIt\n \nallows\n \ndevelopers\n \nto\n \nstructure\n \ncode\n \nusing\n \nclasses\n \nand\n \nobjects\n,\n \nmaking\n \nit\n \nmore\n \nmodular,\n \nreusable,\n \nand\n \nscalable.\n \nThe  core  idea  of  OOPs  is  to  bind  data  and  the  functions  that  operate  on  it,  \npreventing\n \nunauthorized\n \naccess\n \nfrom\n \nother\n \nparts\n \nof\n \nthe\n \ncode.\n \nJava\n \nstrictly\n \nfollows\n \nthe\n \nDRY\n \n(Don't\n \nRepeat\n \nYourself)\n \nPrinciple,\n \nensuring\n \nthat\n \ncommon\n \nlogic\n \nis\n \nwritten\

splitting into chunks 


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
documents=splitter.split_documents(doc)
documents[:5]

[Document(metadata={'producer': 'Skia/PDF m139 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'test 1', 'source': 'test 1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Java  OOP(Object  Oriented  Programming)  \nConcepts\n \nLast  Updated  :  25  Jun,  2025  \n         \nJava  Object-Oriented  Programming  (OOPs)  is  a  fundamental  concept  in  \nJava\n \nthat\n \nevery\n \ndeveloper\n \nmust\n \nunderstand.\n \nIt\n \nallows\n \ndevelopers\n \nto\n \nstructure\n \ncode\n \nusing\n \nclasses'),
 Document(metadata={'producer': 'Skia/PDF m139 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'test 1', 'source': 'test 1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='to\n \nstructure\n \ncode\n \nusing\n \nclasses\n \nand\n \nobjects\n,\n \nmaking\n \nit\n \nmore\n \nmodular,\n \nreusable,\n \nand\n \nscalable.\n \nThe  core  idea  of  OOPs  is  to  bind  data  and  the  functions  that  operate  

In [29]:
from serpapi import GoogleSearch

def search_serpapi(query, api_key):
    params = {
        "q": query[:300],        # Trim the query to 300 chars max
        "api_key": api_key,
        "num": 5                 # Get top 5 search results
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results.get("organic_results", [])


In [30]:
from difflib import SequenceMatcher

def similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()


In [ ]:
SERP_API_KEY = ""  # Replace with your real key
SIMILARITY_THRESHOLD = 0.5 # Tweak this as needed

for i, doc in enumerate(documents):
    chunk_text = doc.page_content.strip()
    print(f"\nChecking Document Chunk {i+1} (first 100 chars):\n{chunk_text[:100]}...")

    try:
        results = search_serpapi(chunk_text, SERP_API_KEY)
    except Exception as e:
        print(f" API Error: {e}")
        continue

    if not results:
        print(" No results returned.")
        continue

    for result in results:
        snippet = result.get("snippet", "")
        score = similarity_score(chunk_text, snippet)

        if score > SIMILARITY_THRESHOLD:
            print("\nPOSSIBLE PLAGIARISM DETECTED:")
            print(f" - Similarity Score: {score:.2f}")
            print(f" - Snippet: {snippet}")
            print(f" - Source Title: {result.get('title', 'N/A')}")
            print(f" - URL: {result.get('link', 'N/A')}")



Checking Document Chunk 1 (first 100 chars):
Java  OOP(Object  Oriented  Programming)  
Concepts
 
Last  Updated  :  25  Jun,  2025  
         
J...

Checking Document Chunk 2 (first 100 chars):
to
 
structure
 
code
 
using
 
classes
 
and
 
objects
,
 
making
 
it
 
more
 
modular,
 
reusable...

Checking Document Chunk 3 (first 100 chars):
from
 
other
 
parts
 
of
 
the
 
code.
 
Java
 
strictly
 
follows
 
the
 
DRY
 
(Don't
 
Repeat
 
...

POSSIBLE PLAGIARISM DETECTED:
 - Similarity Score: 0.66
 - Snippet: Java strictly follows the DRY (Don't Repeat Yourself) Principle, ensuring that common logic is written once (e.g., in parent classes or utility ...
 - Source Title: Java OOP(Object Oriented Programming) Concepts
 - URL: https://www.geeksforgeeks.org/java/object-oriented-programming-oops-concept-in-java/

Checking Document Chunk 4 (first 100 chars):
reused
 
throughout
 
the
 
application.
 
This
 
makes
 
the
 
code:
 
●  Easier  to  maintain:  Ch...

Checking Document Chunk 5

In [34]:
import ollama

def is_ai_generated_local(text, model="llama3"):
    prompt = f"""You are an AI detector. Analyze the text below and classify whether it is:
- AI-generated
- Human-written
- Unclear

Give the one-word verdict followed by a short reason.

Text:
\"\"\"
{text}
\"\"\"
"""
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


In [35]:
for i, doc in enumerate(documents):
    chunk_text = doc.page_content.strip()

    print(f"\nChunk {i+1}")
    judgment = is_ai_generated_local(chunk_text)
    print(f"Local AI Detection Result: {judgment}")



Chunk 1
Local AI Detection Result: **AI-generated**

Reason: The text appears to be a generic, templated introduction to Java OOP concepts, with a focus on formatting and structure rather than content or creativity. The language is straightforward and lacks nuance, while the inclusion of specific dates and details (e.g., "Last Updated") suggests a machine-generated origin. Additionally, the text's brevity and simplicity are consistent with AI-generated content intended to provide a basic overview.

Chunk 2
Local AI Detection Result: Unclear

Reason: The text appears to be a snippet of code or technical documentation, but its structure and formatting are unusual. There is no clear context or coherent sentence structure, making it difficult to determine whether the text was generated by AI or written by a human. Additionally, the use of unnecessary whitespace and inconsistent capitalization may indicate AI-generated text, but it's not conclusive evidence.

Chunk 3
Local AI Detection Res